In [ ]:
import IPython
from IPython.display import Image, Audio, Video
import music21 as ms
from midi2audio import FluidSynth
import tensorflow 
import numpy as np 
import pandas as pd 
from collections import Counter
import sys
from sklearn.model_selection import train_test_split
import warnings
import os
import matplotlib.pyplot as plt
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adamax
import seaborn as sns
from tensorflow.keras.callbacks import ModelCheckpoint
import time

# Loading Files

In [ ]:
#Getting all songs files from directory
def getSongsFromDir(filepath):
    all_songs = []
    for i in os.listdir(filepath):
        if i.endswith(".mid"):
            tr = filepath+i
            midi = ms.converter.parse(tr)
            obj = {
                'name': i.split('.')[0],
                'song': midi
            }

            all_songs.append(obj)
            
    return all_songs

In [ ]:
def removeTempo(song):
    new = ms.stream.Score()
    for part in song:
        if isinstance(part, ms.stream.Part):
            new_p = ms.stream.Part()
            for measure in part:
                if isinstance(measure, ms.stream.Measure):
                    new_m = ms.stream.Measure()
                   
                    for obj in measure:
                        '''
                        if not isinstance(obj, ms.tempo.MetronomeMark):
                            new_m.append(obj)
                        '''
                            
                        if isinstance(obj, ms.tempo.MetronomeMark):
                            quart = obj.getQuarterBPM()
                            quart = round(quart/4, 2)
                            
                            obj.setQuarterBPM(quart)
                        new_m.append(obj)   
                        
                    new_p.append(new_m)

            new.append(new_p)
            
    return new                            

In [ ]:
def createListMusicNotations(score):
    songName = score['name']
    listNotations = []
    
    print('Creating: {}'.format(songName))
    for part in score['song']:
        _clef = 'treble'
        _tempKey = 'C major'
        _tempTimeNumerator = 4
        _tempTimeDenominator = 4
        _measure = 1

        for measure in part:
            _measure = measure.measureNumber
            
            for obj in measure:
                if isinstance(obj, ms.clef.TrebleClef):
                    _clef = 'treble'
                elif isinstance(obj, ms.clef.BassClef):
                    _clef = 'bass'

                elif isinstance(obj, ms.key.Key):
                    _tempKey = obj.name                

                elif isinstance(obj, ms.meter.TimeSignature):
                    _tempTimeNumerator = obj.numerator
                    _tempTimeDenominator = obj.denominator

                elif isinstance(obj, ms.note.Note):
                    _temp = {
                        'songName': songName,
                        'clef': _clef,
                        'keySignature': _tempKey,
                        'timeSignNumerator': _tempTimeNumerator,
                        'timeSignDenominator': _tempTimeDenominator,
                        'measure': _measure,
                        'offset': obj.offset,
                        'duration': obj.quarterLength,
                        'noteName': obj.name,
                        'noteOctave': obj.octave,
                        'chordName': float('nan'),
                        'chordInversion': float('nan'),
                        'chordRoot': float('nan'),
                        'chordThird': float('nan'),
                        'chordFifth': float('nan'),
                        'chordSeventh': float('nan'),
                        'chordBass': float('nan')
                    }
                    listNotations.append(_temp)

                elif isinstance(obj, ms.note.Rest):
                    _temp = {
                        'songName': songName,
                        'clef': _clef,
                        'keySignature': _tempKey,
                        'timeSignNumerator': _tempTimeNumerator,
                        'timeSignDenominator': _tempTimeDenominator,
                        'measure': _measure,
                        'offset': obj.offset,
                        'duration': obj.quarterLength,
                        'noteName': obj.name,
                        'noteOctave': float('nan'),
                        'chordName': float('nan'),
                        'chordInversion': float('nan'),
                        'chordRoot': float('nan'),
                        'chordThird': float('nan'),
                        'chordFifth': float('nan'),
                        'chordSeventh': float('nan'),
                        'chordBass': float('nan')
                    }
                    listNotations.append(_temp)

                elif isinstance(obj, ms.chord.Chord):
                    _temp = {
                        'songName': songName,
                        'clef': _clef,
                        'keySignature': _tempKey,
                        'timeSignNumerator': _tempTimeNumerator,
                        'timeSignDenominator': _tempTimeDenominator,
                        'measure': _measure,
                        'offset': obj.offset,
                        'duration': obj.quarterLength,
                        'noteName': float('nan'),
                        'noteOctave': float('nan'),
                        'chordName': obj.pitchedCommonName,
                        'chordInversion': obj.inversion(),
                        'chordRoot': obj.root(),
                        'chordThird': obj.third,
                        'chordFifth': obj.fifth,
                        'chordSeventh': obj.seventh,
                        'chordBass': obj.bass()
                    }
                    listNotations.append(_temp)
                else:
                    continue


    return listNotations

In [ ]:
def createDataFrame(all_songs):
    lst = []
    for score in all_songs:
        _tempLst = createListMusicNotations(score)

        lst.extend(_tempLst)
        
    df = pd.DataFrame(lst)

    return df

In [ ]:
filepath = "data/__temp/"
all_songs = getSongsFromDir(filepath)
df = createDataFrame(all_songs)

In [ ]:
df.head(5)